In [ ]:
import ipympl
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import comb
from scipy.spatial import ConvexHull

from itertools import pairwise

plt.ioff()
plt.style.use('ggplot')
plt.rcParams.update({'text.usetex': True})
plt.rcParams.update({'font.size': 16})

## Bernstein Polynomials

In [ ]:
def B(i, n, t):
    """
    Calculates the Bernstein polynomial B_{i,n}(t). `n` is the degree.
    """
    if i==0 and n==0:
        return np.ones_like(t)
    elif i < 0 or i > n:
        return np.zeros_like(t)
    else:
        return comb(n, i) * (1 - t)**(n - i) * t**i


def plot_all_B_n(n):
    """
    Plot Bernstein polynomials for i=0,1...n for t in [0,1]. `n` is degree.
    """
    t = np.linspace(0, 1, 101)
    fig, ax = plt.subplots()
    ax.set_xlabel('t')
    ax.set_ylabel(r'$B_{i,n}(t)$')
    ax.set_aspect('equal')
    ax.set_title(r'$B_{{i,n}}(t)$ for $n={0}$'.format(n))
    for i in range(n+1):
        ax.plot(t, B(i, n, t), label=r'$i={0}$'.format(i))
    ax.legend()
    fig.set_tight_layout(True)
    return fig

In [ ]:
fig1 = plot_all_B_n(3)
fig2 = plot_all_B_n(4)
#fig1.savefig('B_3_plot.pdf', bbox_inches='tight')
#fig2.savefig('B_4_plot.pdf', bbox_inches='tight')

In [ ]:
display(fig1.canvas)

In [ ]:
display(fig2.canvas)

## Bezier Curves

In [ ]:
def bezier_curve(P, n, N=101):
    """
    Plot Bezier curve of degree `n` using control points `P`. `P` should be an `ndarray` of
    size (3, n+1).
    """
    assert P.shape[1] == (n + 1)
    t = np.linspace(0, 1, N)
    # B_i_n should have shape (n, N)
    B_i_n = np.vstack([B(i, n, t) for i in range(n+1)])
    return P @ B_i_n

In [ ]:
def plot_bezier_curve(P, n, N=101, control_poly=False, convex_hull=False,
                      title=None):
    """
    Plot the curve optionally with the control polygon and/or the convex hull. 
    P shoule be of shape (3, n+1) where n is the degree.
    """
    dim = P.shape[0]
    if dim != 2:
        raise NotImplementedError('Only 2d curves are supported.')

    curve = bezier_curve(P, n, N)
    coords_list = [curve[i,:] for i in range(dim)]
    # Plot the curve
    fig1, ax1 = plt.subplots()
    ax1.plot(*coords_list, label=r'B\'{e}zier Curve')
    if control_poly:
        # Plot the control polygon
        poly_verts = [P[i,:] for i in range(dim)]
        ax1.plot(*poly_verts, ls='dashed', color='k',
                 label='Control Polygon', marker='o')
    if convex_hull:
        # Plot the convex hull
        hull = ConvexHull(P.T)
        hull_verts = np.append(hull.vertices, hull.vertices[0])
        hull_coords = [P[i, hull_verts] for i in range(dim)]
        ax1.plot(*hull_coords, ls='dotted', label='Convex Hull', color='g')
    if control_poly or convex_hull:
        ax1.legend()
    if title:
        ax1.set_title(title)

    return fig1

In [ ]:
P = np.array([[0, 0], [1, 4], [4, 1], [6, 2]]).T
n = P.shape[1] - 1
fig3 = plot_bezier_curve(P, n, control_poly=True, title='End-point Properties') 
#fig3.savefig('End_points.pdf', bbox_inches='tight')

In [ ]:
display(fig3.canvas)

In [ ]:
P = np.array([[2, 1], [1, 6], [3.5, 2], [4.5, 8], [5, 4], [6, 10], [12, 3]]).T
n = P.shape[1] - 1
fig4 = plot_bezier_curve(P, n, control_poly=True, title='Possible')
display(fig4.canvas)

## de Casteljau's Algorithm

In [ ]:
P0 = np.array([[5,0], [0,1], [1,10], [10,12], [15,6], [11,0]])
n = P0.shape[0] - 1
curve = bezier_curve(P0.T, n)
t0 = 0.4
points = []
points.append(P0)
for i in range(1, n+1):
    temp = np.zeros((n+1-i,2))
    prev = points[i-1]
    for j, k in pairwise(iter(range(n+2-i))):
        temp[j] = (1 - t0)*prev[j] + t0*prev[k]
    points.append(temp)

In [ ]:
fig5, ax5 = plt.subplots()
ax5.plot(curve[0,:], curve[1,:])
ax5.set_aspect('equal')
title = ax5.set_title(r"Iteration 0")
ax5.text(
    13, 11.5, r"$t = {0}$".format(round(t0,1)), ha="center", va="center", size=16,
    bbox=dict(fc='ivory', ec="k", lw=1, alpha=0.5))

for i, verts in enumerate(points):
    ax5.plot(verts[:,0], verts[:,1], marker='o', ls='--', label=f'Iteration {i}')
    #if i==n:
    #    ax5.annotate(r"$P(t={0})$".format(round(t0,1)),
    #                xy=(verts[0,0], verts[0,1]), xycoords='data',
    #                xytext=(verts[0,0]+2, verts[0,1]-2), textcoords='data',
    #                arrowprops=dict(arrowstyle="-|>",
    #                                connectionstyle="arc3",
    #                                color='k'),
    #                )
    #title.set_text(r'Iteration {0}'.format(i))
    #fig5.canvas.draw()
    #fig5.savefig(f'deCasteljau_{i}.pdf', bbox_inches='tight')

title.set_text(r"de Casteljau's algorithm $t={0}$".format(round(t0,1)))
fig5.canvas.draw()
ax5.legend()
display(fig5.canvas)